In [3]:
import AmpliPy
from pathos.pools import ProcessPool

In [9]:
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")

In [4]:
pool = ProcessPool()
def single_variable_func(x):
	return x
results = pool.imap(single_variable_func, [1,2,3,4])
list(results)

[1, 2, 3, 4]

In [5]:
import AmpliPy
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")
# in case you want to look at a specific read type
read = None
for s in in_align:
	read = s
	break
read

In [6]:
# simple, official approach for pickling (using the read string)
import pysam
import dill

header = in_align.header
def recreate_read(read_str):
    return pysam.AlignedSegment.fromstring(read_str, header)

@dill.register(pysam.libcalignedsegment.AlignedSegment)
def save_read(pickler, read):
    pickler.save_reduce(recreate_read, (pysam.AlignedSegment.to_string(read),), obj=read)

dill.dumps(read)

b'\x80\x04\x95)\x03\x00\x00\x00\x00\x00\x00\x8c\ndill._dill\x94\x8c\x10_create_function\x94\x93\x94(h\x00\x8c\x0c_create_code\x94\x93\x94(K\x01K\x00K\x00K\x01K\x04MC C\x0et\x00j\x01\xa0\x02|\x00t\x03\xa1\x02S\x00\x94N\x85\x94(\x8c\x05pysam\x94\x8c\x0eAlignedSegment\x94\x8c\nfromstring\x94\x8c\x06header\x94t\x94\x8c\x08read_str\x94\x85\x94\x8cM/var/folders/pb/tmv6q0ls00g7ks30gfzcpz540000gn/T/ipykernel_8599/1267057380.py\x94\x8c\rrecreate_read\x94K\x06C\x02\x00\x01\x94))t\x94R\x94c__builtin__\n__main__\nh\x0fNN}\x94Nt\x94R\x94X\xf9\x01\x00\x00A00953:313:HFGT7DRXY:2:2122:8666:22795\t161\tNC_045512.2\t1\t50\t58S75M18S\t=\t28255\t28377\tTGCGTCTTGGTTCACCGCTCTCACTCAACATGGCAAGGAAGACCTTAAATTCCCTCGAATTAAAGGTTTACACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACAAACTAAAATGTCTGATA\tFFFF:::FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFF:F:F:FFFFFFFFFFFF\tNM:i:1\tms:i:140\tAS:i:140\tnn:i:0\ttp:A:P\tcm:i:8\ts1

In [74]:
import pysam
import AmpliPy
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")

from pathos.pools import ProcessPool
pool = ProcessPool()
print(pool.ncpus)
def single_variable_func(x):
	return x
results = pool.imap(single_variable_func, in_align)
list(results)

12


 ...]

In [86]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
dill.dumps(read)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('ncalls')
stats.print_stats()

         1703 function calls (1602 primitive calls) in 0.002 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200    0.000    0.000    0.000    0.000 {built-in method builtins.id}
      143    0.000    0.000    0.000    0.000 {method 'write' of '_io.BytesIO' objects}
      141    0.000    0.000    0.000    0.000 /Users/ronakshah/.pyenv/versions/3.8.0/lib/python3.8/pickle.py:241(write)
      125    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
       91    0.000    0.000    0.000    0.000 {built-in method builtins.len}
       89    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
       85    0.000    0.000    0.000    0.000 {method 'tell' of '_io.BytesIO' objects}
       84    0.000    0.000    0.000    0.000 /Users/ronakshah/.pyenv/versions/3.8.0/lib/python3.8/pickle.py:217(commit_frame)
     83/1    0.000    0.000    0.001    0.001 /Users/ronakshah/.pyenv/versions/3.8.0/lib/python3

In [7]:
# another approach to pickling
import pysam
import dill

header = in_align.header
def recreate_read(read_obj):
    return read_obj

@dill.register(pysam.libcalignedsegment.AlignedSegment)
def save_read(pickler, read):
    pickler.save_reduce(recreate_read, ({'cigartuples': read.cigartuples,
 'is_paired': read.is_paired,
 'is_reverse': read.is_reverse,
 'query_alignment_qualities': read.query_alignment_qualities,
 'query_alignment_start': read.query_alignment_start,
 'query_length': read.query_length,
 'reference_end': read.reference_end,
 'reference_start': read.reference_start,
 'template_length': read.template_length},), obj=read)

dill.dumps(read)

b'\x80\x04\x95e\x02\x00\x00\x00\x00\x00\x00\x8c\ndill._dill\x94\x8c\x10_create_function\x94\x93\x94(h\x00\x8c\x0c_create_code\x94\x93\x94(K\x01K\x00K\x00K\x01K\x01MC C\x04|\x00S\x00\x94N\x85\x94)\x8c\x08read_obj\x94\x85\x94\x8cM/var/folders/pb/tmv6q0ls00g7ks30gfzcpz540000gn/T/ipykernel_8599/2136185732.py\x94\x8c\rrecreate_read\x94K\x06C\x02\x00\x01\x94))t\x94R\x94c__builtin__\n__main__\nh\nNN}\x94Nt\x94R\x94}\x94(\x8c\x0bcigartuples\x94]\x94(K\x04K:\x86\x94K\x00KK\x86\x94K\x04K\x12\x86\x94e\x8c\tis_paired\x94\x88\x8c\nis_reverse\x94\x89\x8c\x19query_alignment_qualities\x94h\x00\x8c\t_get_attr\x94\x93\x94h\x00\x8c\x0e_import_module\x94\x93\x94\x8c\x05array\x94\x85\x94R\x94\x8c\x14_array_reconstructor\x94\x86\x94R\x94(h\x1e\x8c\x05array\x94\x93\x94\x8c\x01B\x94K\x00CK%%%%%%%%%\x19%%%%%%%%%%%%%%%%%%%%%%%%%%%\x19%%%%%%%%%%%%%%\x19%%%%%%%%%%%%%%%%%%%%%%\x94t\x94R\x94\x8c\x15query_alignment_start\x94K:\x8c\x0cquery_length\x94K\x97\x8c\rreference_end\x94KK\x8c\x0freference_start\x94K\x00\x8c\

In [83]:
set(["reference_start", "reference_end", "template_length", "query_length", "is_paired", "is_reverse", "cigartuples", "reference_start", "cigartuples", "cigartuples", "reference_start", "is_paired", "is_reverse", "query_length", "cigartuples", "reference_start", "cigartuples", "cigartuples", "query_alignment_qualities", "is_reverse", "cigartuples", "query_alignment_start", "reference_start", "reference_start", "cigartuples", "cigartuples", "cigartuples", "reference_start", "cigartuples", "cigartuples"])

{'cigartuples',
 'is_paired',
 'is_reverse',
 'query_alignment_qualities',
 'query_alignment_start',
 'query_length',
 'reference_end',
 'reference_start',
 'template_length'}

In [8]:
import pysam
import AmpliPy
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")
import pathos
from pathos.pools import ProcessPool
pool = ProcessPool(nodes=pathos.helpers.cpu_count() - 1)

def single_variable_func(x):
	return x

results = set()

for read in in_align:
	results.add(pool.apipe(single_variable_func, read))
	a = set()
	for x in results:
		if x.ready():
			x.get()
		else:
			a.add(x)
	results = a




KeyboardInterrupt: 

In [9]:
import pysam
import AmpliPy
import time
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")

def single_variable_func(x):
	return x
start_time = time.time()
for read in in_align:
	single_variable_func(read)
	
time.time() - start_time

0.24410510063171387

In [15]:
import pysam
import AmpliPy
in_align,_ = AmpliPy.create_AlignmentFile_objects("./example/example_untrimmed_sorted.bam")

from pathos.pools import ProcessPool
pool = ProcessPool()
print(pool.ncpus)
def single_variable_func(x):
	return x
results = pool.imap(single_variable_func, in_align)
list(results)

12


[{'cigartuples': [(4, 58), (0, 75), (4, 18)],
  'is_paired': True,
  'is_reverse': False,
  'query_alignment_qualities': array('B', [37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37]),
  'query_alignment_start': 58,
  'query_length': 151,
  'reference_end': 75,
  'reference_start': 0,
  'template_length': 28377},
 {'cigartuples': [(0, 11), (1, 1), (0, 63), (5, 76)],
  'is_paired': True,
  'is_reverse': False,
  'query_alignment_qualities': array('B', [37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37